# The need for smarter factorization
At this point, the most expensive operations involve factorization.

![Timing](../images/2015-11-02-timing-analysis-new-secular.svg)

1. The kernel rotation should be handled with LU.
2. The Eigendecomposition should be replaced by an efficient SVD.

## 1. Null space basis via LU (replacing QR)

Until now I have been using [SPQR][1] to find a basis for the nullspace of $A$. Now I want to use LU, which is 90% faster.

Wikipedia explains [how to obtain a null space basis using LU][2]. 

[1]: http://faculty.cse.tamu.edu/davis/suitesparse.html
[2]: https://en.wikipedia.org/wiki/Kernel_(linear_algebra)#Computation_by_Gaussian_elimination

In [511]:
# A = sprand(5,10,0.2)
A = sparse([1 0 -3 0 2 -8;
    0 1 5 0 -1 4;
    0 0 0 1 7 -9;
    0 0 0 0 0 0])
A = [spzeros(5,20);sprand(10,20,0.2)]

# """
# Find a basis for the nullspace of A.
# Uses sparse LU.
# """
# function kernel(A)
    m,n = size(A)
dim_n = n - rank(full(A))
# println(dim_n)
    # augment A with identity:
AI = [A;speye(n)]
AI = full(AI)
    # factorize AI':
F = lufact(AI')

# the null basis N comes from the rightmost n-m
# columns of U'. Row operations on AI' correspond
# to column operations on AI, so permuting rows of
# AI' means permuting columns of AI.
U = full(F[:U])'

# N = U[m+1:end,end-dim_n+1:end]
N = U[m+1:end,end-dim_n+1:end]
A*N
# maxabs(A*N)

15x10 Array{Float64,2}:
 0.0       0.0        0.0           0.0       …   0.0          0.0  0.0
 0.0       0.0        0.0           0.0           0.0          0.0  0.0
 0.0       0.0        0.0           0.0           0.0          0.0  0.0
 0.0       0.0        0.0           0.0           0.0          0.0  0.0
 0.0       0.0        0.0           0.0           0.0          0.0  0.0
 0.0       0.0        0.0           0.0       …   0.0          0.0  0.0
 0.0       0.0       -4.16334e-17   0.0          -3.46945e-18  0.0  0.0
 0.0       0.0        0.0           0.0           0.0          0.0  0.0
 0.0       0.0        0.0           0.0           3.03577e-18  0.0  0.0
 0.0       0.0        0.0           0.0          -1.73472e-18  0.0  0.0
 0.310588  0.0        0.0           0.0       …   1.73472e-18  0.0  0.0
 0.0       0.703905  -2.77556e-17   0.0          -3.46945e-18  0.0  0.0
 0.0       0.742088  -0.172029      0.0           0.0          0.0  0.0
 0.689917  0.0       -0.0195292     0.34

In [512]:
dim_n

10

In [508]:
F[:p]

6-element Array{Int32,1}:
 6
 3
 5
 4
 1
 2

In [505]:
Ntest = U[m+1:end,F[:p][end-dim_n+1:end]]

6x3 Array{Float64,2}:
  0.0        0.0   0.0  
  0.0        0.0   0.0  
  0.0        0.0   1.0  
  1.0        0.0   0.0  
 -0.210526   0.0   0.0  
 -0.0526316  1.0  -0.375

In [503]:
A*Ntest

4x3 Array{Float64,2}:
 0.0          -8.0  0.0  
 0.0           4.0  3.5  
 5.55112e-17  -9.0  3.375
 0.0           0.0  0.0  

In [500]:
U

10x6 Array{Float64,2}:
 -8.0   0.0    0.0    0.0         0.0        0.0     
  4.0   3.5    0.0    0.0         0.0        0.0     
 -9.0   3.375  4.75   0.0         0.0        0.0     
  0.0   0.0    0.0    0.0         0.0        0.0     
  0.0   0.0    0.0    0.0         1.0        0.0     
  0.0   0.0    0.0    0.0         0.0        1.0     
  0.0   1.0    0.0    0.0        -0.142857  -0.285714
  0.0   0.0    0.0    1.0         0.0        0.0     
  0.0   0.0    1.0   -0.210526    0.338346   0.203008
  1.0  -0.375  0.25  -0.0526316   0.263158   0.157895

In [478]:
F = lufact(full(A'))
p = invperm(F[:p])
F[:L]

6x4 Array{Float64,2}:
  1.0    0.0        0.0       0.0
  0.375  1.0        0.0       0.0
 -0.25   0.0        1.0       0.0
 -0.0    0.0        0.210526  1.0
 -0.0    0.285714  -0.203008  0.0
 -0.125  0.142857  -0.338346  0.0

In [479]:
p

6-element Array{Int32,1}:
 6
 5
 2
 4
 3
 1

In [495]:
(F[:L]'*(eye(n)[:,p]))'

6x4 Array{Float64,2}:
 -0.125  0.142857  -0.338346  0.0
  0.0    0.285714  -0.203008  0.0
  0.375  1.0        0.0       0.0
  0.0    0.0        0.210526  1.0
 -0.25   0.0        1.0       0.0
  1.0    0.0        0.0       0.0

In [459]:
full(A)

4x6 Array{Int64,2}:
 1  0  -3  0   2  -8
 0  1   5  0  -1   4
 0  0   0  1   7  -9
 0  0   0  0   0   0

In [263]:
F[:p]

20-element Array{Int32,1}:
  6
 10
  3
 12
 17
  2
  8
 11
  7
  5
  1
  4
 13
 14
 16
 15
  9
 18
 19
 20

In [196]:
full((1./F[:Rs]).*F[:U])'

10x6 Array{Float64,2}:
 1.0  0.0   0.0  0.0   0.0   0.0
 0.0  1.0   0.0  0.0   0.0   0.0
 0.0  0.0   1.0  0.0   0.0   0.0
 0.0  0.0   0.0  1.0   0.0   0.0
 0.0  0.0   0.0  0.0   1.0   0.0
 0.0  0.0   0.0  0.0   0.0   1.0
 0.0  0.0   0.0  0.0   0.0   0.0
 0.0  0.0   0.0  1.0   7.0  -9.0
 0.0  1.0   5.0  0.0  -1.0   4.0
 1.0  0.0  -3.0  0.0   2.0  -8.0

In [83]:
Utest = full(F[:U][invperm(F[:p]),invperm(F[:q])])'
Ntest = Utest[m+1:end,end-dim_n+1:end]

6x3 Array{Float64,2}:
 0.0  0.0        0.0      
 0.0  0.0        0.0      
 0.0  0.0        0.0      
 0.5  0.0        0.0      
 0.0  0.0909091  0.0      
 0.0  0.0        0.0454545

In [39]:
?lufact

search: 

```rst
..  lufact(A [,pivot=Val{true}]) -> F

Compute the LU factorization of ``A``. The return type of ``F`` depends on the type of ``A``. In most cases, if ``A`` is a subtype ``S`` of AbstractMatrix with an element type ``T`` supporting ``+``, ``-``, ``*`` and ``/`` the return type is ``LU{T,S{T}}``. If pivoting is chosen (default) the element type should also support ``abs`` and ``<``. When ``A`` is sparse and have element of type ``Float32``, ``Float64``, ``Complex{Float32}``, or ``Complex{Float64}`` the return type is ``UmfpackLU``. Some examples are shown in the table below.

======================= ========================= ========================================
Type of input ``A``     Type of output ``F``      Relationship between ``F`` and ``A``
======================= ========================= ========================================
:func:`Matrix`           ``LU``                   ``F[:L]*F[:U] == A[F[:p], :]``
:func:`Tridiagonal`      ``LU{T,Tridiagonal{T}}`` ``F[:L]*F[:U] == A[F[:p], :]``
:func:`SparseMatrixCSC`  ``UmfpackLU``            ``F[:L]*F[:U] == (F[:Rs] .* A)[F[:p], F[:q]]``
======================= ========================= ========================================

The individual components of the factorization ``F`` can be accessed by indexing:

=========== ======================================= ====== ======================== =============
Component   Description                             ``LU`` ``LU{T,Tridiagonal{T}}`` ``UmfpackLU``
=========== ======================================= ====== ======================== =============
``F[:L]``   ``L`` (lower triangular) part of ``LU``    ✓            ✓                        ✓
``F[:U]``   ``U`` (upper triangular) part of ``LU``    ✓            ✓                        ✓
``F[:p]``   (right) permutation ``Vector``             ✓            ✓                        ✓
``F[:P]``   (right) permutation ``Matrix``             ✓            ✓
``F[:q]``   left permutation ``Vector``                                                      ✓
``F[:Rs]``  ``Vector`` of scaling factors                                                    ✓
``F[:(:)]`` ``(L,U,p,q,Rs)`` components                                                      ✓
=========== ======================================= ====== ======================== =============

================== ====== ======================== =============
Supported function ``LU`` ``LU{T,Tridiagonal{T}}`` ``UmfpackLU``
================== ====== ======================== =============
     ``/``            ✓
     ``\``            ✓                       ✓             ✓
     ``cond``         ✓                                     ✓
     ``det``          ✓                       ✓             ✓
     ``logdet``       ✓                       ✓
     ``logabsdet``    ✓                       ✓
     ``size``         ✓                       ✓
================== ====== ======================== =============
```


lufact lufact!



In [ ]:
"""
Find an orthonormal basis for the nullspace of A.
This matrix may be used to rotate a temporal instanton
problem instance to eliminate all but nullity(A) elements.
"""
function kernel_rotation(A::SparseMatrixCSC{Float64,Int64}; spqr=true)
    m,n = size(A)

    # Assume A always has full row rank of m.
    # It may be possible for this assumption to fail
    # due to numerics, but a rank() check is expensive.
    dim_N = n - m # dimension of nullspace of A

    if spqr
        F = qrfact(sparse(A'))
        # B selects last dim_N cols of Q:
        B = [zeros(size(A,2)-dim_N,dim_N); eye(dim_N)]
        N = sparse(SparseMatrix.SPQR.qmult(SparseMatrix.SPQR.QX, F, SparseMatrix.CHOLMOD.Dense(B)))
        return N
    else
        q = qr(A'; thin=false)[1]
        return q[:,end-dim_N+1:end]
    end
end

In [32]:
full(A)*N

4x3 Array{Float64,2}:
 0.0          0.0   0.0        
 0.0          0.0   2.22045e-16
 5.55112e-17  0.0  -2.22045e-16
 0.0          0.0   0.0        

## Obtaining test matrices
To compare various factorization methods, I will generate some temporal instanton analysis QCQP matrices.

## No more Eig?
Upon studying the structure of the rotated constraint matrix, I believe the Eigendecomposition is unnecessary. I modified my code to eschew the eig. Let's see if I get the same numerical results:

In [1]:
include("../src/TemporalInstanton.jl")
using TemporalInstanton,MatpowerCases

# compile everything with this run:
inputData = load_rts96_data(return_as_type=true);
# Thermal model parameters:
inputData.Tamb = 35. # C
inputData .T0 = 60. #46. # initial line steady-state temp

inputData.time_values = 0:30:300 # five minutes in 30-sec steps
inputData.int_length = 300. # seconds = 5 min
Gp,Dp,Rp = inputData.G0,inputData.D0,inputData.R0
inputData.G0 = [0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp]
inputData.D0 = [0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp]
inputData.R0 = [Rp;1.1*Rp;1.2*Rp;1.3*Rp;1.4*Rp;1.5*Rp]

@time results = solve_temporal_instanton(inputData);

n = length(inputData.k)
nr = length(inputData.Ridx)
T = convert(Int64,length(inputData.G0)/n)
outputData = process_instanton_results(results,n,nr,T,return_as_type=true);

sort(outputData.score)

  likely near /home/jkersulis/.julia/v0.4/GraphLayout/src/draw.jl:3


r=0 check: 	removing 1 lines


 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/jkersulis/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /home/jkersulis/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading /home/jkersulis/.julia/v0.4/GraphLayout/src/draw.jl, in expression starting on line 3
  likely near /home/jkersulis/.julia/v0.4/GraphLayout/src/draw.jl:23
  likely near /home/jkersulis/.julia/v0.4/GraphLayout/src/draw.jl:135


ISF pre-check: 	removing lines Int64[]
 11

104-element Array{Tuple{Float64,Int64},1}:
 (0.263787830020006,25)  
 (0.3273164183285288,30) 
 (1.2886107366327355,69) 
 (2.2041515936832066,89) 
 (2.322973886040895,118) 
 (2.42316154712363,64)   
 (4.67875546231252,42)   
 (4.865240811106734,102) 
 (4.899167201158152,31)  
 (5.678266048785588,80)  
 (5.938620037616521,1)   
 (6.439172990446961,100) 
 (6.4608887584718095,62) 
 ⋮                       
 (178.0855817862111,99)  
 (303.00328573243416,103)
 (370.61443512027347,66) 
 (370.61443512027347,67) 
 (391.3070873721824,33)  
 (428.94449223314234,40) 
 (923.97998538764,72)    
 (979.1988880469704,117) 
 (1319.8524187115077,79) 
 (1390.0270570261027,110)
 (1967.2698882361206,73) 
 (1967.2698882361206,74) 

.471698 seconds (9.28 M allocations: 1.223 GB, 1.74% gc time)


In [2]:
using JLD
Qconstr = load("Qconstr.jld")["Qconstr"][1]
Qeig = load("Qeig.jld")
D,U,Qc,N,A,Qconstr = (Qeig["D"],Qeig["U"],Qeig["Qc"],
                    Qeig["N"],Qeig["A"],Qeig["Qconstr"])
m,n = size(A)
T = 6

([-1.6263e-19,-1.0842e-19,-8.84871e-20,-5.42101e-20,-5.11556e-20,-4.39434e-20,-2.03288e-20,-1.35525e-20,-1.35525e-20,-1.25872e-20  …  1.89735e-19,4.95953e-19,5.42101e-19,1.30104e-18,2.87982e-5,7.68095e-5,0.000204847,0.000546198,0.00145554,0.00387294],
108x108 Array{Float64,2}:
 0.0  0.0  -0.0655642   0.0  -0.250625    …  0.0  0.0  0.0  0.0   0.06375   
 0.0  0.0   0.00899026  0.0  -0.172969       0.0  0.0  0.0  0.0   0.135029  
 0.0  0.0  -0.013932    0.0  -0.0264157      0.0  0.0  0.0  0.0   0.00983275
 0.0  0.0  -0.0648723   0.0   0.172969       0.0  0.0  0.0  0.0   0.0690593 
 0.0  0.0   0.00469648  0.0   0.00552324     0.0  0.0  0.0  0.0   0.00373929
 0.0  0.0  -0.0627797   0.0   0.0195721   …  0.0  0.0  0.0  0.0   0.0721536 
 0.0  0.0   0.122873    0.0   0.254696       0.0  0.0  0.0  0.0  -0.184615  
 0.0  0.0  -0.0115918   0.0  -0.476801       0.0  0.0  0.0  0.0   0.185205  
 0.0  0.0   0.149278    0.0  -0.386489       0.0  0.0  0.0  0.0   0.369547  
 0.0  0.0   0.349904    0.0  

In [136]:
tqr = (@timed qrfact(A'))[2]
tlu = (@timed lufact(A'))[2]

100*(tqr - tlu)/tqr

61.849760215491386

In [47]:
svds(N3,nsv=6)[3]

108x6 Array{Float64,2}:
  0.06375       9.81308e-18   3.92523e-17  …  -1.96262e-17   0.0        
  0.135029     -1.96262e-17  -3.92523e-17     -9.81308e-18   0.0        
  0.00983275    1.96262e-17   0.0             -1.22663e-18  -1.96262e-17
  0.0690593    -9.81308e-18   3.92523e-17     -4.41589e-17   9.81308e-18
  0.00373929    9.81308e-18   0.0              3.06659e-18   9.81308e-18
  0.0721536     0.0          -2.94392e-17  …   0.0           0.0        
 -0.184615      0.0           1.59463e-17      1.96262e-17   1.96262e-17
  0.185205      0.0          -5.39719e-17      1.96262e-17   3.92523e-17
  0.369547      3.92523e-17  -1.07944e-16      1.96262e-17   3.92523e-17
 -0.108246     -3.92523e-17   1.96262e-17      9.81308e-18   1.96262e-17
 -0.378968      3.92523e-17   7.85046e-17  …  -1.96262e-17   0.0        
  0.240085      1.96262e-17  -3.92523e-17     -3.92523e-17   9.81308e-18
 -0.600201     -7.85046e-17   7.85046e-17      1.57009e-16   0.0        
  ⋮                        

In [67]:
Us,Ss,Vs = svds(N3,nsv=6)
Ss = Ss.^2
Ncomplete = N'[:,1:n-m-T]

span = [Ncomplete Vs]

108x108 Array{Float64,2}:
  0.0          0.0         0.0        …   9.81308e-18   0.0        
  0.0          0.0         0.0            7.85046e-17  -3.92523e-17
  0.0          0.0         0.0           -4.90654e-17   9.81308e-18
  0.0          0.0         0.0            3.92523e-17   1.96262e-17
  0.0          0.0         0.0            0.0          -1.22663e-18
  0.0          0.0         0.0        …   2.94392e-17   0.0        
  0.0          0.0         0.0           -5.88785e-17   1.96262e-17
  0.0          0.0         0.0           -1.96262e-17   0.0        
  0.0          0.0         0.0            1.17757e-16   3.92523e-17
  0.0          0.0         0.0            0.0           0.0        
  0.0          0.0         0.0        …  -3.92523e-17  -7.85046e-17
  0.0          0.0         0.0            1.96262e-17   3.92523e-17
  0.0          0.0         0.0           -1.57009e-16   7.85046e-17
  ⋮                                   ⋱                            
  0.104249     0.26333

In [6]:
Ds,Us = eigs(Qconstr,nev=6)

([0.0038729412011626594,0.0014555396811092292,0.0005461976717325809,0.00020484653587438627,7.680946048583108e-5,2.8798246425662575e-5],
108x6 Array{Float64,2}:
  0.06375       1.04083e-17   1.17094e-17  …  -6.93889e-18  -2.77556e-17
  0.135029     -2.01662e-17   1.56125e-17      0.0          -1.38778e-17
  0.00983275   -1.12757e-17   3.46945e-18      1.38778e-17   2.77556e-17
  0.0690593    -5.20417e-18   1.73472e-17      1.38778e-17   1.38778e-17
  0.00373929    8.67362e-19  -2.60209e-18     -1.56125e-17  -2.77556e-17
  0.0721536    -2.1684e-17    2.60209e-18  …   0.0           1.6263e-18 
 -0.184615      5.20417e-17   3.46945e-18      0.0          -2.77556e-17
  0.185205     -7.28584e-17  -1.73472e-17      0.0          -2.77556e-17
  0.369547     -9.36751e-17   6.59195e-17      0.0           1.38778e-17
 -0.108246      6.50521e-17   1.73472e-17      0.0          -6.93889e-18
 -0.378968      2.94903e-17  -6.93889e-18  …   5.55112e-17   9.54098e-18
  0.240085     -1.2837e-16   -2.08167